In [1]:
model_name = 'CGAN'

In [2]:
from keras.layers import Input, Dense, Conv2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import backend as K
from keras.callbacks import Callback
import tensorflow as tf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
import cv2

In [3]:
import random

seed = 42

# Python's built-in random library
random.seed(seed)

# Scikit-learn (only affects some parts of scikit-learn)
from sklearn.utils import check_random_state
check_random_state(seed)

np.random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)

In [4]:
img_rows = 168
img_cols = 168
channels = 1

# Input image dimensions
img_shape = (img_rows, img_cols, channels)

In [5]:
# Set path for data source
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Load Diffusion/dataset/")

Mounted at /content/gdrive


In [6]:
# Set the path to the folder containing saved files
load_path = "./preprocessed_data_168x168_2years"

# Load saved NumPy arrays
X_train = np.load(os.path.join(load_path, 'X_train.npy'))
X_test = np.load(os.path.join(load_path, 'X_test.npy'))
y_train = np.load(os.path.join(load_path, 'y_train.npy'))
y_test = np.load(os.path.join(load_path, 'y_test.npy'))

# Load saved Pandas DataFrames
train_data = pd.read_pickle(os.path.join(load_path, 'train_data.pkl'))
test_data = pd.read_pickle(os.path.join(load_path, 'test_data.pkl'))
metadata = pd.read_pickle(os.path.join(load_path, 'metadata.pkl'))
metadata_original = pd.read_pickle(os.path.join(load_path, 'metadata_original.pkl'))

In [7]:
def merge_dummified_columns(df, dummified_columns, new_column_name):
    """
    Merges dummified columns into a single column in a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the dummified columns.
        dummified_columns (list): A list of the names of the dummified columns.
        new_column_name (str): The name of the new column to create.

    Returns:
        pd.DataFrame: The DataFrame with the dummified columns merged into a single new column.
    """
    # Create a new column containing the column name where the value is 1 for each row
    df[new_column_name] = df[dummified_columns].idxmax(axis=1)

    # Drop the dummified columns
    df.drop(dummified_columns, axis=1, inplace=True)

    return df

test_metadata = metadata_original.loc[test_data.columns]

test_metadata = merge_dummified_columns(test_metadata,
                     dummified_columns=['chilledwater', 'electricity', 'gas', 'hotwater', 'steam'],
                     new_column_name='meter_type')
test_metadata = merge_dummified_columns(test_metadata,
                     dummified_columns=['Education', 'Entertainment/public assembly', 'Lodging/residential', 'Office', 'Public services'],
                     new_column_name='building_type')

test_metadata


,lat,lng,year,meter_type,building_type
electricity_Lamb_office_Caitlin_2016,51.497838,-3.186246,2016,electricity,Office
electricity_Rat_public_Kelle_2016,38.903504,-77.005349,2016,electricity,Public services
electricity_Rat_assembly_Ezequiel_2016,38.903504,-77.005349,2016,electricity,Entertainment/public assembly
electricity_Bear_assembly_Roxy_2016,37.871903,-122.260729,2016,electricity,Entertainment/public assembly
gas_Shrew_office_Rose_2017,51.499840,-0.124663,2017,gas,Office
...,...,...,...,...,...
steam_Moose_education_Omar_2016,45.421500,-75.697200,2016,steam,Education
electricity_Rat_public_Chrissy_2017,38.903504,-77.005349,2017,electricity,Public services
electricity_Hog_office_Merilyn_2016,44.978782,-93.255398,2016,electricity,Office
steam_Bull_assembly_Amalia_2016,30.267200,-97.743100,2016,steam,Entertainment/public assembly


In [8]:
model_path = "./weights_168x168"

# List all files in the directory
all_files = os.listdir(model_path)

# Filter files that contain 'model'
model_files = [file for file in all_files if model_name in file]

for model_file in model_files:
  print(model_file)
  #loaded_model = load_model(os.path.join(model_path, model_files[-1]))

CGAN_model_8000.keras
CGAN_model_9000.keras
CGAN_model_10000.keras
CGAN_model_11000.keras
CGAN_model_12000.keras
CGAN_model_13000.keras
CGAN_model_14000.keras
CGAN_model_15000.keras
CGAN_model_16000.keras
CGAN_model_1000.keras
CGAN_model_2000.keras
CGAN_model_3000.keras
CGAN_model_4000.keras
CGAN_model_5000.keras
CGAN_model_6000.keras
CGAN_model_7000.keras


In [9]:
loaded_model = load_model(os.path.join(model_path, 'CGAN_model_16000.keras'))
loaded_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 13)]                 0         []                            
                                                                                                  
 input_13 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 dense_12 (Dense)            (None, 100)                  1400      ['input_14[0][0]']            
                                                                                                  
 multiply_2 (Multiply)       (None, 100)                  0         ['input_13[0][0]',            
                                                                     'dense_12[0][0]']      

In [10]:
# Function to display images
def display_images(images, title):
    plt.figure(figsize=(10, 1))
    for i in range(10):
        plt.subplot(1, 10, i+1)
        plt.imshow(images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

In [11]:
# Create a directory to save the generated image datasets
model_path = "./generation/"+model_name+"_168x168"
if not os.path.exists(model_path):
    os.makedirs(model_path)

labels_to_condition = y_test  # This assumes you have already prepared y_test

# Loop to create 30 different datasets
for i in range(1, 31):
    np.random.seed(i)  # Set the random seed

    z_dim = 100

    # Sample random noise
    z = np.random.normal(0, 1, (labels_to_condition.shape[0], z_dim))

    labels_to_generate = y_test

    # Generate images from random noise
    generated_images = loaded_model.predict([z, labels_to_generate], verbose=0)

    # Rescale image pixel values to [0, 1]
    generated_images = 0.5 * generated_images + 0.5

    # Show the first 10 images of the generated dataset
    display_images(generated_images, title=f"Dataset with Seed {i}")

    # Save the dataset
    np.save(os.path.join(model_path, f"generated_images_seed_{i}.npy"), generated_images)

Output hidden; open in https://colab.research.google.com to view.